<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>


<a href="http://www.hupi.fr/" ><img src="http://www.hupi.fr/wp-content/uploads/2016/03/hupi_logo_vectoris_menu.png" style="float:right; max-width: 300px; display: inline"  alt="Wikistat"/></a>


</center>

# [Ateliers: Technologies des grosses données](https://github.com/wikistat/Ateliers-Big-Data)

# Recommandation de Films par Filtrage Collaboratif: [NMF](http://wikistat.fr/pdf/st-m-explo-nmf.pdf) de la librairie [MLlib](http://spark.apache.org/mllib/) de <a href="http://spark.apache.org/"><img src="http://spark.apache.org/images/spark-logo-trademark.png" style="max-width: 100px; display: inline" alt="Spark"/></a>

## 1. Introduction

Ce calepin traite d'un problème classique de recommandation par filtrage collaboratif en utilisant les ressources de la librairie [MLlib de Spark]([http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.ALS) avec l'API pyspark. Le problème général est décrit en [introduction](https://github.com/wikistat/Ateliers-Big-Data/tree/master/3-MovieLens) et dans une [vignette](http://wikistat.fr/pdf/st-m-datSc3-colFil.pdf) de [Wikistat](http://wikistat.fr/). Il est appliqué aux données publiques du site [GroupLens](http://grouplens.org/datasets/movielens/). L'objectif est de tester les méthodes et la procédure d'optimisation sur le plus petit jeu de données composé de 100k notes  de 943 clients sur 1682 films où chaque client a au moins noté 20 films. Les jeux de données plus gros (1M, 10M, 20M notes) peuvent être utilisés pour "passer à l'échelle volume". 

Ce calepin s'inspire des exemples de la [documentation](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.ALS) et d'un [tutoriel](https://github.com/jadianes/spark-movie-lens/blob/master/notebooks/building-recommender.ipynb) de [Jose A. Dianes](https://www.codementor.io/jadianes). Le sujet a été traité lors d'un [Spark Summit](https://databricks-training.s3.amazonaws.com/movie-recommendation-with-mllib.html).

L'objectif est d'utiliser ces seules données pour proposer des recommandations.  Les données initiales sont sous la forme d'une matrice **très creuse** (*sparse*) contenant des notes ou évaluations. **Attention**, les "0" de la matrice ne sont pas des notes mais des *données manquantes*, le film n'a pas encore été vu ou évalué. 

Un algorithme satisfaisant à l'objectif de *complétion de grande matrice creuse*, et implémenté dans un logiciel libre d'accès est disponible dans la librairie [softImpute de R](https://cran.r-project.org/web/packages/softImpute/index.html). SOn utilisaiton est décrite dans un autre [calepin](https://github.com/wikistat/Ateliers-Big-Data/blob/master/3-MovieLens/Atelier-MovieLens-softImpute.ipynb). La version de [NMF](http://wikistat.fr/pdf/st-m-explo-nmf.pdf) de [MLlib de Spark](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.ALS) autorise permet également la complétion.

En revanche,la version  de NMF incluse dans la librairie [Scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html) traite également des [matrices creuses](http://docs.scipy.org/doc/scipy/reference/sparse.html) mais le critère (moindres carrés) optimisé considère les "0" comme des notes nulles, pas comme des données manquantes. *Elle n'est pas adaptée au problème de complétion*, contrairement à celle de MLliB. Il faudrait sans doute utiliser la librairie [nonnegfac](https://github.com/kimjingu/nonnegfac-python) en Python  de [Kim et al. (2014)](http://link.springer.com/content/pdf/10.1007%2Fs10898-013-0035-4.pdf); **à tester**!

Dans la première partie, le plus petit fichier est partagé en trois échantillons: apprentissage, validation et test; l'optimisation du rang de la factorisation (nombre de facteurs latents) est réalisée par minimisation de l'erreur estimée sur l'échantillon de validation.

Ensuite le plus gros fichier est utilisé pour évaluer l'impact de la taille de la base d'apprentissage.

## 2 Importation des données en HDFS
Les données doivent être stockées à un emplacement accessibles de tous les noeuds du cluster pour permettre la construction de la base de données réparties (RDD). Dans une utilisation monoposte (*standalone*) de *Spark*, elles sont simplement chargées dans le répertoire courant. 

In [ ]:
# Chargement des fichiers si ce n'est déjà fait
import urllib fichier réduit
f = urllib.urlretrieve("http://www.math.univ-toulouse.fr/~besse/Wikistat/data/ml-ratings100k.csv","ml-ratings100k.csv")

Les données sont lues comme une seule ligne de texte avant d'être restructurées au bon format d'une *matrice creuse* à savoir une liste de triplets contenant les  indices de ligne, de colonne et la note pour les seules valeurs renseignées.

In [1]:
# Importer les données au format texte dans un RDD

# le chemin dépend de l'environnement
path=""
small_ratings_raw_data = sc.textFile(path+"ml-ratings100k.csv")
# Identifier et afficher la première ligne
small_ratings_raw_data_header = small_ratings_raw_data.take(1)[0]
small_ratings_raw_data_header

NameError: name 'sc' is not defined

In [2]:
# Séparer les champs (user, item, note) dans un nouveau RDD
# La première ligne est éliminée
# .cache() : le RDD est conservé en mémoire une fois traité
small_ratings_data = small_ratings_raw_data.filter(lambda line: line!=small_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()
# Trois premières lignes pour voir
small_ratings_data.take(3)

NameError: name 'small_ratings_raw_data' is not defined

## 3. Optimisation du rang sur l'échantillon 10k
Le fichier comporte 10 000 évaluations croisant les avis de mille utilisateurs sur les films qu'ils ont vus parmi 1700.

### 3.1 Constitution des échantillons

Séparation aléatoire en trois échantillons apprentissage, validation et test. Le paramètre de rang est optimisé en minimisant l'estimaiton de l'erreur sur l'échantillon test. Cette stratégie, plutôt qu'ue validation croisée est plus adaptée à des données massives.


In [ ]:
tauxApp=0.6
tauxVal=0.2
tauxTes=0.2
# Si le total est inférieur à 1, les données sont sous-échantillonnées.
trainRDD, validRDD, testRDD = small_ratings_data.randomSplit([6, 2, 2], seed=0L)
# validation et test à prédire, sans les notes
validRDD_P = validRDD.map(lambda x: (x[0], x[1]))
testRDD_P = testRDD.map(lambda x: (x[0], x[1]))

### 3.2 Optimisation du rang de la NMF

L'erreur d'imputation des données, donc de recommandation, est estimée sur l'échantillon de validation pour différentes valeurs (grille) du rang de la factorisation matricielle. 

Il faudrait en principe aussi optimiser la valeur du paramètre de pénalisation pris à 0.1 par défaut.

*Point important:* l'erreur d'ajustement de la factorisation ne prend en compte que les valeurs listées dans la matrice creuses, pas les "0" qui sont des données manquantes.

In [ ]:
from pyspark.mllib.recommendation import ALS
import math
# Initialisation du générateur
seed = 5L
# Nombre max d'itérations (ALS)
iterations = 10
# Régularisation L1; à optimiser également
regularization_parameter = 0.1
# Choix d'une grille pour les valeurs du rang à optimiser
ranks = [4, 8, 12]
# Une erreur par rang
errors = [0, 0, 0]
# Initialisations
err = 0
tolerance = 0.02
min_error = float('inf')
best_rank = -1
best_iteration = -1

In [ ]:
# Estimation pour chaque valeur de rang
for rank in ranks:
    model = ALS.train(trainRDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    # Prévision de l'échantillon de validation
    predRDD = model.predictAll(validRDD_P).map(lambda r: ((r[0], r[1]), r[2]))
    # Jointure avec la vraie solution (échantillon de validation)
    notETpredRDD = validRDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predRDD)
    # Calcul du RMSE
    error = math.sqrt(notETpredRDD.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print 'Pour le rang %s le RMSE est: %s' % (rank, error)
    if error < min_error:
        min_error = error
        best_rank = rank
# Meilleure solution
print 'Rang optimal: %s' % best_rank

### 3.3 Résultats et test

In [ ]:
# Quelques prévisions
predRDD.take(3)

In [ ]:
# "vraie" note et sa prévision
notETpredRDD.take(3)

Prévision finale de l'échantillon test.

**Remarque :** il aurait été judicieux de fusionner les échantillons d'apprentissage et de validation avant de réestimer le modèle avec le rang optimal précédemment trouvé avant de prévoir l'échantillon test. 

In [ ]:
# Le seul échantillon d'apprentissage est utilisé avec le rang optimal
model = ALS.train(trainRDD, best_rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
# Prévision de l'échantillon test
predRDD = model.predictAll(testRDD_P).map(lambda r: ((r[0], r[1]), r[2]))
# Jointure
notETpredRDD = testRDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predRDD)
# Calcul de l'erreur.
error = math.sqrt(notETpredRDD.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print 'RMSE pour le test: %s' % (error)

## 3 Analyse du fichier complet

MovieLens propose un plus gros fichier avec 20M de notes (138000 utilisateurs, 27000 films). Ce fichier est utilisé pour extraire un fichier test de deux millions de notes à reconstruire. Les paramètres précédemment optimisés, ils pourraient sans doute l'être mieux, sont appliqués pour une succesion d'estimation / prévision avec une taille croissante de l'échantillon d'apprentissage. Il aurait été plus élégant d'automatiser le travail dans une boucle mais lorsque les données sont les plus volumineuses des comportement mal contrôlés de Spark peuvent provoquer des plantages par défaut de mémoire.

### 3.1 Lecture des données

Le fichier est prétraité de manière analogue.

In [ ]:
# Chargement des fichiers si ce n'est déjà fait
import urllib
# fichier complet mais compressé
f = urllib.urlretrieve("http://www.math.univ-toulouse.fr/~besse/Wikistat/data/ml-ratings20M.zip","ml-ratings20M.zip")

In [ ]:
# Importer les données au format texte dans un RDD
path=""
ratings_raw_data = sc.textFile(path+"ml-ratings20M.zip")
# Identifier et afficher la première ligne
ratings_raw_data_header = ratings_raw_data.take(1)[0]
ratings_raw_data_header

In [ ]:
# Séparer les champs (user, item, note) dans un nouveau RDD
# La première ligne est éliminée
# les trois premiers champs sont sélectionnés
ratings_data = ratings_raw_data.filter(lambda line: line!=ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2]))
# Trois premières lignes pour voir
ratings_data.take(3)

### 3.2 Echantillonnage

Extraction de l'échantillon test et éventuellement sous-échantillonnage de l'échantillon d'apprentissage. 

In [ ]:
tauxTest=0.10
(trainDataTot, testData) = ratings_data.randomSplit([1-tauxTest, tauxTest], seed=0L)
testData.count()

In [ ]:
# départ

In [ ]:
# Sous-échantillonnage de l'apprentissage permettant de 
# tester pour des tailles croissantes de cet échantillon
tauxEch=1
(trainData, DropData) = trainDataTot.randomSplit([tauxEch, 1-tauxEch])
trainData.count()

### 3.3 Estimation du modèle

Le modèle est estimé en utilisant les valeurs des paramètres obtenues dans l'étape précédente.

In [ ]:
from pyspark.mllib.recommendation import ALS
import math
import time
time_start=time.time()
# Initialisation du générateur
seed = 5L
# Nombre max d'itérations (ALS)
iterations = 10
# Régularisation L1 (valeur par défaut)
regularization_parameter = 0.1
tolerance = 0.02
min_error = float('inf')
best_rank = 8
# Estimation pour chaque valeur de rang
model = ALS.train(trainData, rank=best_rank, seed=seed, 
                iterations=iterations,lambda_=regularization_parameter)
time_end=time.time()
time_als=(time_end - time_start)
print("ALS prend %d s" %(time_als)) 

### 3.4 Prévision de l'échantillon test et erreur

In [ ]:
# Listes (i,j) des notes à prédire
testData_r = testData.map(lambda x: (x[0], x[1]))
# Prévision de l'échantillon test
predTest = model.predictAll(testData_r).map(lambda r: ((r[0], r[1]), r[2]))
# Jointure
notETpred = testData.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predTest)

# Calcul de l'erreur
erreur = math.sqrt(notETpred.map(lambda r: (r[1][0] - r[1][1])**2).mean())
print 'RMSE pour le test: %s' % (erreur)

Quelques résultats montrant l'évolution du temps de calcul et de l'erreur de prévision en fonction de la taille de l'échantillon d'apprentissage. Le nombre d'utilisateurs de la plateforme étant assez aléatoire, les temps calculés sont peu fiables. D'autre part il est probable que la valeur des paramètres optimaux dépendent de la taille de l'échantillon d'apprenitssage.

Taille | Temps(s) | RMSE
-------|-------|------
217439 | 70    | 1.65
1029416| 73    | 1.06
2059855| 72    | 1.05
4119486| 89    | 0.88
6176085| 99    | 0.85
10301909| 117  | 0.83
12361034| 125  | 0.83
14414907| 137  | 0.82
16474087| 148  | 0.818
18538142| 190  | 0.816
20596263| 166  | 0.82
